# Basic sqlalchemy example

This example shows how to connect to the postgresql database previously created (see https://github.com/BONSAMURAIS/schema). To connect, we use the python package sqlalchemy (https://www.sqlalchemy.org/) because it allows pythonic database manipulation.

First off, imports.

In [1]:
import subprocess
import sqlalchemy as sqla
import sqlalchemy.ext.automap
import sqlalchemy.orm

Let's start the local postgresql server.

In [2]:
# start psql server
try:
    r = subprocess.check_output("pg_ctl status")
except subprocess.CalledProcessError as e:
    subprocess.call("pg_ctl -l logfile start")

Now that the server is started, we can connect to it. We use the user name and database name previously defined in the schema.

In [3]:
# connect to database
user = (enter superuser name here)
password = ""
url = "localhost"
port = "5432"
database = "bonsai"
db = sqla.create_engine("postgresql://"+user+":"+password+"@"+url+":"+port+"/"+database+"", echo=True)

The big strength of sqlalchemy is that it allows database manipulation in a very pythonic way, without the need to enter sql strings. For this to work, we bind classes to each of the tables in the database. The following code reads the existing tables from the database and creates one class variable for each table.

In [4]:
# get database metadata
meta = sqla.MetaData()
meta.reflect(db)

# create one class for each table in the database
Base = sqlalchemy.ext.automap.automap_base(metadata=meta)
Base.prepare()
for table_name in meta.tables.keys():
    exec(table_name + "= Base.classes." + table_name)
    
# create a session
session = sqla.orm.sessionmaker(bind=db)()

2020-03-02 13:50:44,394 INFO sqlalchemy.engine.base.Engine select version()
2020-03-02 13:50:44,397 INFO sqlalchemy.engine.base.Engine {}
2020-03-02 13:50:44,403 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-03-02 13:50:44,404 INFO sqlalchemy.engine.base.Engine {}
2020-03-02 13:50:44,407 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-02 13:50:44,409 INFO sqlalchemy.engine.base.Engine {}
2020-03-02 13:50:44,413 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-03-02 13:50:44,414 INFO sqlalchemy.engine.base.Engine {}
2020-03-02 13:50:44,415 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-03-02 13:50:44,416 INFO sqlalchemy.engine.base.Engine {}
2020-03-02 13:50:44,419 INFO sqlalchemy.engine.base.Engine SELECT c.relname FROM pg_class c JOIN pg_namespace n ON n.oid = c.relnamespace WHERE n.nspname = %(schema)s AND c.relkind in ('r', 'p')
2020-0

2020-03-02 13:50:44,544 INFO sqlalchemy.engine.base.Engine {'table_name': 'datasource'}
2020-03-02 13:50:44,546 INFO sqlalchemy.engine.base.Engine 
            SELECT a.attname,
              pg_catalog.format_type(a.atttypid, a.atttypmod),
              (SELECT pg_catalog.pg_get_expr(d.adbin, d.adrelid)
                FROM pg_catalog.pg_attrdef d
               WHERE d.adrelid = a.attrelid AND d.adnum = a.attnum
               AND a.atthasdef)
              AS DEFAULT,
              a.attnotnull, a.attnum, a.attrelid as table_oid,
              pgd.description as comment
            FROM pg_catalog.pg_attribute a
            LEFT JOIN pg_catalog.pg_description pgd ON (
                pgd.objoid = a.attrelid AND pgd.objsubid = a.attnum)
            WHERE a.attrelid = %(table_oid)s
            AND a.attnum > 0 AND NOT a.attisdropped
            ORDER BY a.attnum
        
2020-03-02 13:50:44,547 INFO sqlalchemy.engine.base.Engine {'table_oid': 16676}
2020-03-02 13:50:44,552 INFO sqlalc

2020-03-02 13:50:44,627 INFO sqlalchemy.engine.base.Engine {'table_oid': 16642}
2020-03-02 13:50:44,632 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                cons.conkey as key,
                a.attnum as col_num,
                a.attname as col_name
            FROM
                pg_catalog.pg_constraint cons
                join pg_attribute a
                  on cons.conrelid = a.attrelid AND
                    a.attnum = ANY(cons.conkey)
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'u'
        
2020-03-02 13:50:44,633 INFO sqlalchemy.engine.base.Engine {'table_oid': 16642}
2020-03-02 13:50:44,637 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                pg_get_constraintdef(cons.oid) as src
            FROM
                pg_catalog.pg_constraint cons
            WHERE
                cons.conrelid = %(table_oid)s AND

2020-03-02 13:50:44,701 INFO sqlalchemy.engine.base.Engine {'table': 16697}
2020-03-02 13:50:44,707 INFO sqlalchemy.engine.base.Engine 
              SELECT
                  i.relname as relname,
                  ix.indisunique, ix.indexprs, ix.indpred,
                  a.attname, a.attnum, c.conrelid, ix.indkey::varchar,
                  ix.indoption::varchar, i.reloptions, am.amname
              FROM
                  pg_class t
                        join pg_index ix on t.oid = ix.indrelid
                        join pg_class i on i.oid = ix.indexrelid
                        left outer join
                            pg_attribute a
                            on t.oid = a.attrelid and a.attnum = ANY(ix.indkey)
                        left outer join
                            pg_constraint c
                            on (ix.indrelid = c.conrelid and
                                ix.indexrelid = c.conindid and
                                c.contype in ('p', 'u', 'x')

2020-03-02 13:50:44,765 INFO sqlalchemy.engine.base.Engine 
            SELECT t.typname as "name",
               pg_catalog.format_type(t.typbasetype, t.typtypmod) as "attype",
               not t.typnotnull as "nullable",
               t.typdefault as "default",
               pg_catalog.pg_type_is_visible(t.oid) as "visible",
               n.nspname as "schema"
            FROM pg_catalog.pg_type t
               LEFT JOIN pg_catalog.pg_namespace n ON n.oid = t.typnamespace
            WHERE t.typtype = 'd'
        
2020-03-02 13:50:44,768 INFO sqlalchemy.engine.base.Engine {}
2020-03-02 13:50:44,771 INFO sqlalchemy.engine.base.Engine 
            SELECT t.typname as "name",
               -- no enum defaults in 8.4 at least
               -- t.typdefault as "default",
               pg_catalog.pg_type_is_visible(t.oid) as "visible",
               n.nspname as "schema",
               e.enumlabel as "label"
            FROM pg_catalog.pg_type t
                 LEFT JOIN pg_cat

2020-03-02 13:50:44,830 INFO sqlalchemy.engine.base.Engine {'table_oid': 16797}
2020-03-02 13:50:44,833 INFO sqlalchemy.engine.base.Engine 
        SELECT conname
           FROM  pg_catalog.pg_constraint r
           WHERE r.conrelid = %(table_oid)s AND r.contype = 'p'
           ORDER BY 1
        
2020-03-02 13:50:44,836 INFO sqlalchemy.engine.base.Engine {'table_oid': 16797}
2020-03-02 13:50:44,839 INFO sqlalchemy.engine.base.Engine 
          SELECT r.conname,
                pg_catalog.pg_get_constraintdef(r.oid, true) as condef,
                n.nspname as conschema
          FROM  pg_catalog.pg_constraint r,
                pg_namespace n,
                pg_class c

          WHERE r.conrelid = %(table)s AND
                r.contype = 'f' AND
                c.oid = confrelid AND
                n.oid = c.relnamespace
          ORDER BY 1
        
2020-03-02 13:50:44,840 INFO sqlalchemy.engine.base.Engine {'table': 16797}
2020-03-02 13:50:44,845 INFO sqlalchemy.engine.base.E

2020-03-02 13:50:44,897 INFO sqlalchemy.engine.base.Engine 
            SELECT a.attname,
              pg_catalog.format_type(a.atttypid, a.atttypmod),
              (SELECT pg_catalog.pg_get_expr(d.adbin, d.adrelid)
                FROM pg_catalog.pg_attrdef d
               WHERE d.adrelid = a.attrelid AND d.adnum = a.attnum
               AND a.atthasdef)
              AS DEFAULT,
              a.attnotnull, a.attnum, a.attrelid as table_oid,
              pgd.description as comment
            FROM pg_catalog.pg_attribute a
            LEFT JOIN pg_catalog.pg_description pgd ON (
                pgd.objoid = a.attrelid AND pgd.objsubid = a.attnum)
            WHERE a.attrelid = %(table_oid)s
            AND a.attnum > 0 AND NOT a.attisdropped
            ORDER BY a.attnum
        
2020-03-02 13:50:44,898 INFO sqlalchemy.engine.base.Engine {'table_oid': 16718}
2020-03-02 13:50:44,902 INFO sqlalchemy.engine.base.Engine 
            SELECT t.typname as "name",
               pg_catal

2020-03-02 13:50:44,948 INFO sqlalchemy.engine.base.Engine {'table_oid': 16797}
2020-03-02 13:50:44,951 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                pg_get_constraintdef(cons.oid) as src
            FROM
                pg_catalog.pg_constraint cons
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'c'
        
2020-03-02 13:50:44,952 INFO sqlalchemy.engine.base.Engine {'table_oid': 16797}
2020-03-02 13:50:44,955 INFO sqlalchemy.engine.base.Engine 
            SELECT
                pgd.description as table_comment
            FROM
                pg_catalog.pg_description pgd
            WHERE
                pgd.objsubid = 0 AND
                pgd.objoid = %(table_oid)s
        
2020-03-02 13:50:44,955 INFO sqlalchemy.engine.base.Engine {'table_oid': 16797}
2020-03-02 13:50:44,959 INFO sqlalchemy.engine.base.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_

2020-03-02 13:50:45,010 INFO sqlalchemy.engine.base.Engine {'table_oid': 16740}
2020-03-02 13:50:45,015 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                cons.conkey as key,
                a.attnum as col_num,
                a.attname as col_name
            FROM
                pg_catalog.pg_constraint cons
                join pg_attribute a
                  on cons.conrelid = a.attrelid AND
                    a.attnum = ANY(cons.conkey)
            WHERE
                cons.conrelid = %(table_oid)s AND
                cons.contype = 'u'
        
2020-03-02 13:50:45,016 INFO sqlalchemy.engine.base.Engine {'table_oid': 16740}
2020-03-02 13:50:45,019 INFO sqlalchemy.engine.base.Engine 
            SELECT
                cons.conname as name,
                pg_get_constraintdef(cons.oid) as src
            FROM
                pg_catalog.pg_constraint cons
            WHERE
                cons.conrelid = %(table_oid)s AND

2020-03-02 13:50:45,075 INFO sqlalchemy.engine.base.Engine {'table_oid': 16773}
2020-03-02 13:50:45,078 INFO sqlalchemy.engine.base.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            AND c.relname = %(table_name)s AND c.relkind in
            ('r', 'v', 'm', 'f', 'p')
        
2020-03-02 13:50:45,079 INFO sqlalchemy.engine.base.Engine {'table_name': 'balancable_property'}
2020-03-02 13:50:45,082 INFO sqlalchemy.engine.base.Engine 
            SELECT a.attname,
              pg_catalog.format_type(a.atttypid, a.atttypmod),
              (SELECT pg_catalog.pg_get_expr(d.adbin, d.adrelid)
                FROM pg_catalog.pg_attrdef d
               WHERE d.adrelid = a.attrelid AND d.adnum = a.attnum
               AND a.atthasdef)
              AS DEFAULT,
              a.attnotnull, a.attnum, a.attrelid as table_oid,
         

That's all the setup we need. Now we can begin manipulating the database. First, let's see which tables/classes we have.

In [5]:
[t for t in meta.tables.keys()]

['license',
 'datasource',
 'location',
 'agent',
 'activity',
 'activity_type',
 'flow',
 'flow_object',
 'unit',
 'temporal_extent',
 'reference_unit',
 'balancable_property']

Next, let's get the first 10 entries of the location table:

In [13]:
[l.__dict__ for l in session.query(location).order_by(location.id)[0:10]]

2020-03-02 13:52:53,175 INFO sqlalchemy.engine.base.Engine SELECT location.id AS location_id, location.label AS location_label, location.identifier AS location_identifier, location.uri AS location_uri 
FROM location ORDER BY location.id 
 LIMIT %(param_1)s
2020-03-02 13:52:53,176 INFO sqlalchemy.engine.base.Engine {'param_1': 10}


[{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x293801ec198>,
  'uri': 'https://www.geonames.org/countries/ID/',
  'label': 'Indonesia',
  'id': 1,
  'identifier': 'ID',
  'activity_collection': [],
  'agent_collection': [],
  'datasource_collection': []},
 {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x293801ec208>,
  'uri': 'https://www.geonames.org/countries/MY/',
  'label': 'Malaysia',
  'id': 2,
  'identifier': 'MY'},
 {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x293801ec278>,
  'uri': 'https://www.geonames.org/countries/CL/',
  'label': 'Chile',
  'id': 3,
  'identifier': 'CL'},
 {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x293801ec2e8>,
  'uri': 'https://www.geonames.org/countries/BO/',
  'label': 'Bolivia',
  'id': 4,
  'identifier': 'BO'},
 {'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x293801ec358>,
  'uri': 'https://www.geonames.org/countries/PE/',
  'label': 'Peru',
  'id': 5,
  'identi